# Implementing "ReAct: Synergizing Reasoning and Acting in Language Models"

Based on Sam Witteveen's tutorial [YouTube URL](https://youtu.be/Eug2clsLtFs?si=UbTmeEattxh6wAG7), [Google Colab](https://colab.research.google.com/drive/1ValkiKnB1RUXU8QKYB-W8S__C4AWJDCb?usp=sharing#scrollTo=Z87JdRqudKQr)

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI

In [2]:
load_dotenv(find_dotenv())

True

## CoT (Chain of Thought) Reasoning

In [3]:
llm = OpenAI(
    model='text-davinci-003',
    temperature=0,
    max_tokens=256,
)

In [6]:
text = 'Explain step by step. Current year is 2023. How old is the president of the United States?'
response = llm(text)
print(response)



1. Determine the current president of the United States. As of 2023, the current president of the United States is Joe Biden.

2. Determine Joe Biden's date of birth. Joe Biden was born on November 20, 1942.

3. Calculate Joe Biden's age. Joe Biden's age in 2023 is 80 years old.


## Issue

Without CoT, any statement after the initial thought is reinforcing the initial statement.

Prone to hallucination.

In [7]:

text = 'How would I get from Singapore to San Francisco?'
response = llm(text)
print(response)



The most direct route from Singapore to San Francisco is to fly. You can find direct flights from Singapore to San Francisco on airlines such as Singapore Airlines, United Airlines, and American Airlines.


In [8]:
text = 'Explain step by step. How would I get from Singapore to San Francisco?'
response = llm(text)
print(response)



1. Book a flight from Singapore to San Francisco. You can do this online or through a travel agent.

2. Pack your bags and check in at the airport. Make sure you have all the necessary documents for travel, such as a valid passport and visa.

3. Board the plane and fly from Singapore to San Francisco. Depending on the airline and route, the flight may take anywhere from 15 to 24 hours.

4. Upon arrival in San Francisco, go through customs and immigration.

5. Collect your luggage and take a taxi or public transportation to your destination.


In [9]:
# Question -> Thought -> Action -> Observation

question = "How old is the president of the United States?"

manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

In [10]:
# The tool itself is a hallucination. The prompt "Question -> Thought -> Action -> Observation" is never forced. 
print(llm(manual_react))


Thought: I need to search the president of the United States, find their age, then answer the question.
Action: Search[president of the United States]
Observation: Joe Biden is the 46th and current president of the United States.
Thought: Joe Biden is the president of the United States. I need to search Joe Biden and find his age.
Action: Search[Joe Biden]
Observation: Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician who is the 46th and current president of the United States.
Thought: Joe Biden was born in 1942, so he is 78 years old.
Action: Finish[78 years old]


In [11]:
from langchain.docstore import Wikipedia
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents.react.base import DocstoreExplorer

In [13]:
docstore = DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name='Search',
        func=docstore.search,
        description='useful for when you need to ask with search',
    ),
    Tool(
        name='Lookup',
        func=docstore.lookup,
        description='useful for when you need to ask with lookup',
    ),
]
llm = OpenAI(model='text-davinci-003', temperature=0)
react = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.REACT_DOCSTORE, 
    verbose=True,
)

In [14]:
question = 'How old is the president of the United States?'
response = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search the president of the United States, find their age, then find the answer.
Action: Search[president of the United States]
Observation: The president of the United States (POTUS) is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the leader of the nation with the largest

In [15]:
print(response)

46


In [16]:
question = 'How old will the president of the United States be in September 2023?'
response = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search the president of the United States, find their date of birth, then calculate their age in September 2023.
Action: Search[president of the United States]
Observation: The president of the United States (POTUS) is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, with notable expansion during the presidency of Franklin D. Roosevelt. In modern times, the president is one of the world's most powerful political figures – the leader of the only remaining global superpower. As the leade

In [17]:
print(response)

80 years old


In [19]:
print(react.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

## Multi-hop example

In [23]:
import langchain

langchain.debug = True

In [24]:
question = 'What is the first film that Leonardo DiCaprio won an Oscar for, and who directed that movie?'
response = react.run(question)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the first film that Leonardo DiCaprio won an Oscar for, and who directed that movie?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the first film that Leonardo DiCaprio won an Oscar for, and who directed that movie?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought

In [25]:
print(response)

The Revenant (2015), Martin Scorsese
